### Постановка задачи
Продолжим нашу работу в службе безопасности. Разобравшись с тем, какие предикторы могут помогать нам предсказывать запрещенный багаж, давайте применим наши знания для повышения безопасности в аэропорту. Обучим наш алгоритм различать запрещенный и разрешенный багаж на уже имеющихся данных и применим его для сканирования нового багажа.

Напишите функцию, которая принимает на вход два набора данных. Первый dataframe содержит информацию об уже осмотренном багаже (запрещенный или нет, вес, длина, ширина, тип сумки).

Второй набор данных — это информация о новом багаже, который сканируется прямо сейчас. В данных также есть информация:  вес, длина, ширина, тип сумки и имя пассажира (смотри описание переменных в примере).

Используя первый набор данных, обучите регрессионную модель различать запрещенный и разрешенный багаж. При помощи полученной модели для каждого наблюдения в новых данных предскажите вероятность того, что багаж является запрещенным. Пассажиров, чей багаж получил максимальное значение вероятности, мы попросим пройти дополнительную проверку.

Итого, ваша функция принимает два набора данных и возвращает имя пассажира с наиболее подозрительным багажом. Если несколько пассажиров получили максимальное значение вероятности, то верните вектор с несколькими именами.

В этой задаче для предсказания будем использовать все предикторы, даже если некоторые из них оказались незначимыми. Для предсказания стройте модель без взаимодействия предикторов.

**Данные**
Обучающий датасет: https://stepic.org/media/attachments/course/524/test_data_passangers.csv
Датасет для предсказания: https://stepic.org/media/attachments/course/524/predict_passangers.csv

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sf
import statsmodels.api as sm

In [2]:
training_data = pd.read_csv('https://stepic.org/media/attachments/course/524/test_data_passangers.csv')
training_data['is_prohibited'] = np.where((training_data.is_prohibited == 'No'), 0, training_data.is_prohibited)
training_data['is_prohibited'] = np.where((training_data.is_prohibited == 'Yes'), 1, training_data.is_prohibited)
training_data.head(10)

,is_prohibited,weight,length,width,type
0,0,81,49,13,Suitcase
1,0,72,49,25,Suitcase
2,0,79,60,22,Suitcase
3,0,89,49,24,Suitcase
4,0,87,54,13,Suitcase
5,0,91,42,25,Suitcase
6,0,74,54,17,Suitcase
7,0,76,49,22,Suitcase
8,0,74,49,12,Suitcase
9,0,84,53,26,Suitcase


In [12]:
glm_binomial = sf.glm('is_prohibited ~ type + weight + length + width', training_data, family=sm.families.Binomial()).fit()
glm_binomial.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            Generalized Linear Model Regression Results                             
====================================================================================================
Dep. Variable:     ['is_prohibited[0]', 'is_prohibited[1]']   No. Observations:                   30
Model:                                                  GLM   Df Residuals:                       25
Model Family:                                      Binomial   Df Model:                            4
Link Function:                                        Logit   Scale:                          1.0000
Method:                                                IRLS   Log-Likelihood:                -13.701
Date:                                      Mon, 10 Oct 2022   Deviance:                       27.402
Time:                                              15:17:30   Pearson chi2:                     28.4
No. Iterations:                                           5   Pseudo R-squ. (CS):             0.3768
Covariance Type:                                  nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            2.8485      9.224      0.309      0.757     -15.230      20.927
type[T.Suitcase]     2.4760      1.074      2.305      0.021       0.371       4.581
weight               0.0578      0.091      0.634      0.526      -0.121       0.237
length              -0.1254      0.099     -1.264      0.206      -0.320       0.069
width               -0.1033      0.104     -0.996      0.319      -0.307       0.100
====================================================================================
"""

In [13]:
coef = sf.glm('is_prohibited ~ type + weight + length + width', training_data, family=sm.families.Binomial()).fit().params
coef

Intercept           2.848451
type[T.Suitcase]    2.476006
weight              0.057797
length             -0.125409
width              -0.103319
dtype: float64

In [14]:
# Suitcase = 1, Bag = 0
# Получаем модель:
# log(odds) = 2.848451 + (2.476006 * T.Suitcase) + (0.057797 * weight) + (-0.125409 * length) + (-0.103319 * width)

In [15]:
df_predict = pd.read_csv('https://stepic.org/media/attachments/course/524/predict_passangers.csv')
df_predict['type'] = np.where((df_predict.type == 'Bag'), 0, df_predict.type)
df_predict['type'] = np.where((df_predict.type == 'Suitcase'), 1, df_predict.type)
df_predict.head()

,weight,length,width,type,passangers
0,81,56,24,1,Bob
1,80,47,18,0,Anatoliy
2,76,54,20,0,Ivan
3,87,59,19,0,Polina
4,80,59,19,1,Velimir


In [16]:
df_predict['log_odds'] = coef[0] + (coef[1] * df_predict['type']) + (coef[2] * df_predict['weight']) + (coef[3] * df_predict['length']) + (coef[4] * df_predict['width'])
df_predict.head()

,weight,length,width,type,passangers,log_odds
0,81,56,24,1,Bob,0.503403
1,80,47,18,0,Anatoliy,-0.281802
2,76,54,20,0,Ivan,-1.597492
3,87,59,19,0,Polina,-1.485457
4,80,59,19,1,Velimir,0.585972


In [17]:
# Считаем шансы
df_predict['odds'] = [np.exp(i) for i in df_predict['log_odds']]
df_predict.head()

,weight,length,width,type,passangers,log_odds,odds
0,81,56,24,1,Bob,0.503403,1.654342
1,80,47,18,0,Anatoliy,-0.281802,0.754423
2,76,54,20,0,Ivan,-1.597492,0.202404
3,87,59,19,0,Polina,-1.485457,0.226399
4,80,59,19,1,Velimir,0.585972,1.796737


In [18]:
# Считаем вероятность
df_predict['probability'] = df_predict['odds'] / (1 + df_predict['odds'])
df_predict.head()

,weight,length,width,type,passangers,log_odds,odds,probability
0,81,56,24,1,Bob,0.503403,1.654342,0.623259
1,80,47,18,0,Anatoliy,-0.281802,0.754423,0.430012
2,76,54,20,0,Ivan,-1.597492,0.202404,0.168332
3,87,59,19,0,Polina,-1.485457,0.226399,0.184605
4,80,59,19,1,Velimir,0.585972,1.796737,0.642440


In [19]:
# Находим пассажиров
passengers = [df_predict['passangers'][i] for i in range(len(df_predict['passangers']))
                  if df_predict['probability'][i] == df_predict['probability'].max()]
passengers

['Vsevolod']